In [36]:
import numpy as np
import yfinance as yf
import pandas as pd
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import pandas_datareader.data as web
from datetime import timedelta
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn_som.som import SOM


In [37]:
def compute_regimes(start_date, end_date, k=8):
    start_date = '1960-01-01'
    end_date = '2022-10-10'

    nasdaq = web.DataReader('NASDAQCOM', "fred", start_date, end_date).dropna().resample('W').last() 
    nasdaq2= nasdaq.pct_change(periods=4)[1:] # compute weekly returns
    infl=web.DataReader('CPIAUCSL', "fred", start_date, end_date).dropna()
    infl2=infl.pct_change(periods=12)[1:] # inflation wrt prior year
    infl3=infl2.dropna().resample('W').mean().interpolate() # resample to weekly data
    int_rate=web.DataReader('DFF', "fred", start_date, end_date).dropna()
    int_rate2=int_rate.resample('W').mean() # resample to weekly data


    #sent=web.DataReader('UMCSENT', "fred", start_date, end_date).dropna()
    #sent2=sent#.pct_change(periods=1)[1:]
    #sent3=sent2.dropna().resample('W').mean().interpolate()/4

    aux=nasdaq2
    aux['INFLATION']=infl3
    aux['INTEREST_R']=int_rate2
    #aux['SENTIMENT'] = sent3
    aux=aux.dropna()

    scaler = StandardScaler() # scale data to std normal
    scaler.fit(aux)
    norm_data=pd.DataFrame(scaler.transform(aux),index=aux.index,columns=aux.columns)

    som = SOM(m=k, n=1, dim=norm_data.shape[1]) # Self-Organising Map
    som.fit(norm_data.to_numpy())
    hs = som.predict(norm_data.to_numpy())
    st = pd.DataFrame(hs,index=norm_data.index)
    data = pd.concat([norm_data,st],axis=1,sort='False').dropna()
    data.rename(columns={0:'REGIME'}, inplace=True)
    return data
data=compute_regimes(8)


/var/folders/5r/3sxxysys3kb_pxzpp4npjd9h0000gn/T/ipykernel_57090/3398874954.py:36: FutureWarning: Passing non boolean values for sort is deprecated and will error in a future version!
  data = pd.concat([norm_data,st],axis=1,sort='False').dropna()


In [38]:
px.line(data,title='Normalized data and corresponding regimes')

In [40]:
data

,NASDAQCOM,INFLATION,INTEREST_R,REGIME
DATE,,,,
1971-03-07,0.451379,0.154332,-0.370359,3
1971-03-14,0.260265,0.130559,-0.302760,3
1971-03-21,0.714089,0.106786,-0.254118,3
1971-03-28,0.480786,0.083014,-0.307768,3
1971-04-04,0.404239,0.059241,-0.191168,3
...,...,...,...,...
2022-08-07,1.360356,1.435643,-0.636821,3
2022-08-14,2.245874,1.433388,-0.636821,3
2022-08-21,1.114893,1.431133,-0.636821,3
